# LSTM Stock Predictor Using Closing Prices

## Data Preparation

In this activity, we will prepare the training and testing data for the LSTM model.

We will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the `X` and `y` values
4. Reshape the `X_train` and `X_test` data for the model.

**Note:** The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [1]:
# Initial imports
import numpy as np
import pandas as pd
from path import Path

%matplotlib inline

In [2]:
# Set the random seed for reproducibility
# Note: This is used for model prototyping, but it is good practice to comment this out and run multiple experiments to evaluate your model.
from numpy.random import seed

seed(1)
from tensorflow import random

random.set_seed(2)

### Data Loading

In this activity, we will use closing prices from different stocks to make predictions of future closing prices based on the temporal data of each stock.

In [3]:
# Load the stocks data
df = pd.read_csv(
    Path("../Resources/stock_data.csv"),
    index_col="date",
    infer_datetime_format=True,
    parse_dates=True,
)
df.head()

,VIX,Gold,T-Bonds,Junk Bonds,Oil
date,,,,,
2014-10-20,97500.0,119.80,29.18,121.08,33.78
2014-10-21,83000.0,120.02,29.14,121.59,34.17
2014-10-22,95250.0,119.34,29.01,120.75,33.53
2014-10-23,84750.0,118.52,28.96,120.84,34.37
2014-10-24,82750.0,118.35,28.96,121.32,34.17


### Creating the Features `X` and Target `y` Data

The first step towards preparing the data is to create the input features vectors `X` and the target vector `y`. We will use the `window_data()` function to create these vectors.

This function chunks the data up with a rolling window of _X<sub>t</sub> - window_ to predict _X<sub>t</sub>_.

The function returns two `numpy` arrays:

* `X`: The input features vectors.

* `y`: The target vector.

The function has the following parameters:

* `df`: The original DataFrame with the time series data.

* `window`: The window size in days of previous closing prices that will be used for the prediction.

* `feature_col_number`: The column number from the original DataFrame where the features are located.

* `target_col_number`: The column number from the original DataFrame where the target is located.

In [4]:
def window_data(df, window, feature_col_number, target_col_number):
    """
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    """
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In the forthcoming activities, we will predict closing prices using a `5` days windows of previous _T-Bonds_ closing prices, so that, we will create the `X` and `y` vectors by calling the `window_data` function and defining a window size of `5` and setting the features and target column numbers to `2` (this is the column with the _T-Bonds_ closing prices).

In [5]:
# Creating the features (X) and target (y) data using the window_data() function.
window_size = 5

feature_column = 2
target_column = 2
X, y = window_data(df, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[29.18 29.14 29.01 28.96 28.96]
 [29.14 29.01 28.96 28.96 29.06]
 [29.01 28.96 28.96 29.06 29.1 ]
 [28.96 28.96 29.06 29.1  28.88]
 [28.96 29.06 29.1  28.88 28.82]] 

y sample values:
[[29.06]
 [29.1 ]
 [28.88]
 [28.82]
 [28.59]]


### Splitting Data Between Training and Testing Sets

To avoid the dataset being randomized, we will manually split the data using array slicing.

In [6]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split - 1]
y_test = y[split:]

### Scaling Data with `MinMaxScaler`

Once the training and test datasets are created, we need to scale the data before training the LSTM model. We will use the `MinMaxScaler` from `sklearn` to scale all values between `0` and `1`.

Note that we scale both features and target sets.

In [7]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)

y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

### Reshape Features Data for the LSTM Model

The LSTM API from Keras needs to receive the features data as a _vertical vector_, so that we need to reshape the `X` data in the form `reshape((X_train.shape[0], X_train.shape[1], 1))`.

Both sets, training, and testing are reshaped.

In [8]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.8973747 ]
  [0.88782816]
  [0.85680191]
  [0.84486874]
  [0.84486874]]

 [[0.88782816]
  [0.85680191]
  [0.84486874]
  [0.84486874]
  [0.86873508]]

 [[0.85680191]
  [0.84486874]
  [0.84486874]
  [0.86873508]
  [0.87828162]]

 [[0.84486874]
  [0.84486874]
  [0.86873508]
  [0.87828162]
  [0.82577566]]

 [[0.84486874]
  [0.86873508]
  [0.87828162]
  [0.82577566]
  [0.81145585]]] 

X_test sample values:
[[[0.96897375]
  [0.97136038]
  [0.96181384]
  [0.93078759]
  [0.87828162]]

 [[0.97136038]
  [0.96181384]
  [0.93078759]
  [0.87828162]
  [0.82100239]]

 [[0.96181384]
  [0.93078759]
  [0.87828162]
  [0.82100239]
  [0.81861575]]

 [[0.93078759]
  [0.87828162]
  [0.82100239]
  [0.81861575]
  [0.78042959]]

 [[0.87828162]
  [0.82100239]
  [0.81861575]
  [0.78042959]
  [0.78281623]]]


---

## Build and Train the LSTM RNN

In this section, we will design a custom LSTM RNN in Keras and fit (train) it using the training data we defined.

We will need to:

1. Define the model architecture in Keras.

2. Compile the model.

3. Fit the model to the training data.

### Importing the Keras Modules

To build an LSTM RNN model in Keras, we will use the `Sequential` model as we did before; however, we will have two new types of layers:

* `LSTM`: It's used to add an LSTM layer to the model.

* `Dropout`: Dropout is a regularization technique for reducing overfitting in neural networks. This type of layer applies the dropout technique to the input.

In [11]:
# Import required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

### Defining the LSTM RNN Model Structure

To create an LSTM RNN model, we will add `LSTM` layers. The `return_sequences` parameter needs to set to `True` every time we add a new `LSTM` layer, excluding the final layer. The `input_shape` is the number of time steps and the number of indicators

After each `LSTM` layer, we add a `Dropout` layer to prevent overfitting. The parameter passed to the `Dropout` layer is the fraction of nodes that will be drop on each epoch, for this demo, we will use a dropout value of `0.2`, it means that on each epoch we will randomly drop `20%` of the units.

The number of units in each `LSTM` layers, is equal to the size of the time window, in this demo, we are taking five previous `T-Bons` closing price to predict the next closing price.

In [12]:
(X_train.shape[1], 1)

(5, 1)

In [13]:
# Define the LSTM RNN model.
model = Sequential()

number_units = 5
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units = number_units,
    return_sequences = True,
    input_shape = (X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(
    units = number_units,
    return_sequences=True
))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(
    units = number_units,
    return_sequences=False
))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))


### Compiling the LSTM RNN Model

We will compile the model, using the `adam` optimizer, as loss function, we will use `mean_square_error` since the value we want to predict is continuous.

In [14]:
# Compile the model
model.compile(
    optimizer = "adam",
    loss="mean_squared_error" #Regression
) 

In [16]:
140/5

28.0

In [15]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 5)              140       
_________________________________________________________________
dropout (Dropout)            (None, 5, 5)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 5)              220       
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dropout_2 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6

### Training the Model

Once the model is defined, we train (fit) the model using `10` epochs. Since we are working with time-series data, it's important to set `shuffle=False` since it's necessary to keep the sequential order of the data.

We can experiment with the `batch_size` parameter; however, smaller batch size is recommended; in this demo, we will use a `batch_size=1`.

In [17]:
# Train the model
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Train on 876 samples
Epoch 1/10
876/876 [==============================] - 15s 17ms/sample - loss: 0.0438
Epoch 2/10
876/876 [==============================] - 10s 11ms/sample - loss: 0.0225
Epoch 3/10
876/876 [==============================] - 10s 11ms/sample - loss: 0.0203
Epoch 4/10
876/876 [==============================] - 10s 11ms/sample - loss: 0.0194
Epoch 5/10
876/876 [==============================] - 10s 12ms/sample - loss: 0.0163
Epoch 6/10
876/876 [==============================] - 12s 14ms/sample - loss: 0.0151
Epoch 7/10
876/876 [==============================] - 15s 17ms/sample - loss: 0.0121
Epoch 8/10
876/876 [==============================] - 11s 12ms/sample - loss: 0.0115
Epoch 9/10
876/876 [==============================] - 11s 12ms/sample - loss: 0.0106
Epoch 10/10
876/876 [==============================] - 11s 13ms/sample - loss: 0.0115
